In [17]:
from os import path, listdir
from sklearn.model_selection import train_test_split
import numpy as np
import skimage.io


In [11]:
def dice(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score

    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [12]:
test_png = './wdata/train_png'
test_png2 = './wdata/train_png_5_3_0'
test_png3 = './wdata/train_png_pan_6_7'
masks_dir = './wdata/masks'

pred_folder = './wdata/test_pred'

all_files = []
for f in listdir(test_png):
    if '.png' in f:
        all_files.append(path.join(test_png, f))

train_idxs0, test_idxs = train_test_split(np.arange(len(all_files)), test_size=0.1, random_state=8)
test_files = [all_files[i] for i in list(test_idxs)]



In [27]:
thresh = 130
dice_msk = []
dice_msk_speed0 = []
dice_msk_speed1 = []
dice_msk_speed2 = []
dice_msk_speed_cont = []

i = 0
for file in test_files:
    print(i)
    test_gt_msk = skimage.io.imread(file.replace('train_png', 'masks'))
    test_gt_msk_speed0 = skimage.io.imread(file.replace('train_png', 'masks').replace('.png','_speed0.png'))
    test_gt_msk_speed1 = skimage.io.imread(file.replace('train_png', 'masks').replace('.png','_speed1.png'))
    test_gt_msk_speed2 = skimage.io.imread(file.replace('train_png', 'masks').replace('.png','_speed2.png'))
    test_gt_msk_speed_cont = skimage.io.imread(file.replace('train_png', 'masks').replace('.png','_speed_cont.png'))

    test_pred_msk = np.where(skimage.io.imread(file.replace('train_png', 'test_pred'))[6:1306,6:1306]>thresh, 255, 0)
    test_pred_msk_speed0 = np.where(skimage.io.imread(file.replace('train_png', 'test_pred').replace('.png','_speed0.png'))[6:1306,6:1306]>thresh, 255, 0)
    test_pred_msk_speed1 = np.where(skimage.io.imread(file.replace('train_png', 'test_pred').replace('.png','_speed1.png'))[6:1306,6:1306]>thresh, 255, 0)
    test_pred_msk_speed2 = np.where(skimage.io.imread(file.replace('train_png', 'test_pred').replace('.png','_speed2.png'))[6:1306,6:1306]>thresh, 255, 0)
    test_pred_msk_speed_cont = np.where(skimage.io.imread(file.replace('train_png', 'test_pred').replace('.png','_speed_cont.png'))[6:1306,6:1306]>thresh, 255, 0)
    
    dice_msk.append(dice(test_pred_msk,test_gt_msk))
    dice_msk_speed0.append(dice(test_pred_msk_speed0,test_gt_msk_speed0))
    dice_msk_speed1.append(dice(test_pred_msk_speed1,test_gt_msk_speed1))
    dice_msk_speed2.append(dice(test_pred_msk_speed2,test_gt_msk_speed2))
    dice_msk_speed_cont.append(dice(test_pred_msk_speed_cont,test_gt_msk_speed_cont))
    i +=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236


In [28]:
dice_msk = np.array(dice_msk)
dice_msk_speed0 = np.array(dice_msk_speed0)
dice_msk_speed1 = np.array(dice_msk_speed1)
dice_msk_speed2 = np.array(dice_msk_speed2)
dice_msk_speed_cont = np.array(dice_msk_speed_cont)



In [31]:
dice_msk.mean(),dice_msk_speed0.mean(),dice_msk_speed1.mean(),dice_msk_speed2.mean(),dice_msk_speed_cont.mean()

(0.678855231522259,
 0.3926156833715194,
 0.46397524628644016,
 0.6005781082009635,
 0.33268758986180325)